## Black-Scholes-Merton Model

6 Key Parameters:

* S: Current Underlying Price
* K: Strike Price
* T: Time to Maturity (in Years)
* r: Risk-Free Interest Rate
* $\sigma$: Volatility of Underlying stock
* q: Dividend Yield

### Call Option Value:

$$ C = Se^{(-qT)}N(d_1) - Ke^{(-rT)}N(d_2) $$
$$ P = Ke^{(-rT)}N(d_2) - Se^{(-qT)}N(d_1) $$

---

$$d_1 = \frac{ln(S/K) + (r - q + 0.5\sigma^2)T}{\sigma \sqrt T}$$
$$d_2 = d_1 - \sigma \sqrt T $$

---

In [1]:
import jax.numpy as jnp
from jax import grad
from jax.scipy.stats import norm as jnorm

#reference package
import blackscholes as bs

In [ ]:
def black_scholes(S, K, T, r, sigma, q=0, otype='call'):
    d1 = (jnp.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * jnp.sqrt(T))
    d2 = d1 - sigma * jnp.sqrt(T)
    if otype == 'call':
        call = S * jnp.exp(-q * T) * jnorm.cdf(d1, 0, 1) - K * jnp.exp(-r * T) * jnorm.cdf(d2, 0, 1)
        return call
    else:
        put = K * jnp.exp(-r * T) * jnorm.cdf(-d2, 0, 1) - S * jnp.exp(-q * T) * jnorm.cdf(-d1, 0, 1)
        return put

In [30]:
S = 100.0  # Current stock price
K = 100.0  # Strike price
T = 1.0    # Time to expiration in years
r = 0.05   # Risk-free interest rate
sigma = 0.2  # Volatility of the underlying stock
q = 0.0    # Dividend yield

#testing the function
call_price = black_scholes(S, K, T, r, sigma, q, otype='call')
print("Call Price (JAX):", call_price)

# Using the reference package to validate
call = bs.BlackScholesCall(S, K, T, r, sigma)
print("Call Price (Reference):", call.price())

put_price = black_scholes(S, K, T, r, sigma, q, otype='put')
print("Put Price (JAX):", put_price)

# Using the reference package to validate
put = bs.BlackScholesPut(S, K, T, r, sigma)
print("Put Price (Reference):", put.price())

Call Price (JAX): 10.450577
Call Price (Reference): 10.450583572185565
Put Price (JAX): 5.5735245
Put Price (Reference): 5.573526022256971


### Greeks:

1. Delta: Option Price Change due to change in Underlying Price Change.
    
    * $\Delta$ = $\frac{\delta C}{\delta S}$

2. Gamma: Option Price Change due to change in Volatility.

    * $\Gamma$ = $\frac{\delta ^2 C}{\delta S^2}$

3. Theta: Option Price Change due to changes in Time to Maturity.

    * $\Theta$ = $\frac{\delta C}{\delta T}$

4. Rho: Option Price Change due to change in Interest Rate.

    * $\rho$ = $\frac{\delta C}{\delta r}$

5. Vega: Option Price Change due to change in Volatility.

    * $\nu$ = $\frac{\delta C}{\delta \sigma}$

In [25]:
delta = grad(black_scholes, argnums=0)
gamma = grad(delta, argnums=0)
theta = grad(black_scholes, argnums=2)
rho = grad(black_scholes, argnums=3)
vega = grad(black_scholes, argnums=4)

In [31]:
our_delta = delta(S, K, T, r, sigma, q, otype='call')
print("Delta (JAX):", our_delta)
print("Delta (Reference):", call.delta())
our_gamma = gamma(S, K, T, r, sigma, q, otype='call')
print("Gamma (JAX):", our_gamma)
print("Gamma (Reference):", call.gamma())
our_theta = -theta(S, K, T, r, sigma, q, otype='call')
print("Theta (JAX):", our_theta)
print("Theta (Reference):", call.theta())
our_rho = rho(S, K, T, r, sigma, q, otype='call')
print("Rho (JAX):", our_rho)
print("Rho (Reference):", call.rho())
our_vega = vega(S, K, T, r, sigma, q, otype='call')
print("Vega (JAX):", our_vega)
print("Vega (Reference):", call.vega())

Delta (JAX): 0.6368306
Delta (Reference): 0.6368306511756191
Gamma (JAX): 0.018762019
Gamma (Reference): 0.018762017345846895
Theta (JAX): -6.414028
Theta (Reference): -6.414027546438197
Rho (JAX): 53.232483
Rho (Reference): 53.232481545376345
Vega (JAX): 37.524036
Vega (Reference): 37.52403469169379


In [32]:
def loss(S, K, T, r, sigma_guess, price, q=0, otype="call"):

    #Price with GUESS for Volatility
    theoretical_price = black_scholes(S, K, T, r, sigma_guess, q, otype)

    market_price = price

    return theoretical_price - market_price

loss_grad = grad(loss, argnums=4)

### Newton-Ralphson Method

$x^g$ is the guess for volatility.

$x^g_{n+1}$ = $x^g_n - \frac{f(x^g_n)}{f'(x^g_n)}$

$f(x^g_n)$ is the loss function. This is the difference between:
* $P_{theory}$: Theoretical Price (E.g. Black-Scholes valuation using the guess for volatility)
* $P_{actual}$: Actual Price (E.g. Market Price)

$f'(x^g_n)$ is the gradient of the loss function.

More explicitly, it's the partial derivative of the loss function with respect to the volatility:
$$\frac{\delta \mathcal{L}}{\delta \sigma}$$

Where $\mathcal{L}$ is our loss function, as defined above.

So in terms of our situation, we hava:
$$\sigma _{n+1} = \sigma _n - \frac{\mathcal{L}(\sigma _n)}{\frac{\delta \mathcal{L}}{\delta \sigma}}$$

We can solve this iteratively! Here is the algorithm:
1. Make a guess for the volatility.
2. Calculate the loss function.
3. Check if the loss is less than the tolerance, $\epsilon$
* $|P_{theory} - P_{actual}| < \epsilon$
    * If yes, STOP!
    * If no, CONTINUE to Step 4.
4. Calculate the gradient of the loss funtion.
5. Update the Volatility using the Newton-Raphson formula.
6. Repeat steps 2-5 unitl converged, or we hit the maximum number of iterations, $N_{iter}$.

In [33]:
S = 100.0
K = 110.0
T = 1.0
r = 0.05
sigma = 0.2
q = 0
otype = "call"

price = black_scholes(S, K, T, r, sigma, q, otype)
print(f"Theoretical Price: {price:.2f}")

Theoretical Price: 6.04


\textbf{Note:} Price was computed for a volatility of 0.2.

This means that when we use this price and the above parameters

(as well as a bad guess for the volatility!)

We should expect the solver to converge to $\sigma$ = 0.2!

In [41]:
def solve_for_iv(S, K, T, r, price, sigma_guess = 1.8, q=0, otype="call", N_iter = 20, epsilon = 0.001, verbose = True):
    #1. Guess for Volatility
    sigma = sigma_guess
    for i in range(N_iter):
        print("\nIteration:", i)
        #2. calculate loss function
        loss_val = loss(S, K, T, r, sigma, price, q, otype)

        if verbose:
            print("Current Error in Theoretical vs Market Price:")
            print(loss_val)
        
        #3. Check if loss is less than the tolerance

        #If yes, STOP!
        if abs(loss_val) < epsilon:
            break

        #If no, CONTINUE to Step 4.
        else:

            #4. Calculate gradient of loss function.
            loss_grad_val = loss_grad(S, K, T, r, sigma, price, q, otype)

            #5. Update the volatility using the Newton-Raphson formula
            sigma = sigma - loss_val / loss_grad_val
    
    return sigma

calculated_iv = solve_for_iv(S, K, T, r, price, sigma_guess=1.8, q=q, otype=otype)
print(f"Calculated Implied Volatility: {calculated_iv:.4f}")
print(f"Actual Implied Volatility: {sigma:.4f}")


Iteration: 0
Current Error in Theoretical vs Market Price:
56.310246

Iteration: 1
Current Error in Theoretical vs Market Price:
-19.478813

Iteration: 2
Current Error in Theoretical vs Market Price:
0.7546234

Iteration: 3
Current Error in Theoretical vs Market Price:
0.0011482239

Iteration: 4
Current Error in Theoretical vs Market Price:
7.6293945e-06
Calculated Implied Volatility: 0.2000
Actual Implied Volatility: 0.2000


In [42]:
print(calculated_iv)

0.20000018
